In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


In [ ]:
from pyspark.sql import Row
data = [
    Row(CountryCode=1, Name='USA', Year=30 , Value=1),
    Row(CountryCode=2, Name='India', Year=25, Value=1),
    Row(CountryCode=3, Name='Russia', Year=35, Value=1)
]

df = spark.createDataFrame(data)
df.show()

+-----------+------+----+-----+
|CountryCode|  Name|Year|Value|
+-----------+------+----+-----+
|          1|   USA|  30|    1|
|          2| India|  25|    1|
|          3|Russia|  35|    1|
+-----------+------+----+-----+



In [ ]:
df2 = spark.read.options(delimiter='|', inferschema=True, header=True).csv("/content/file.txt")
df2.show()

AnalysisException: Path does not exist: file:/content/file.txt

In [ ]:
df2.printSchema()

In [ ]:
df2.columns

In [ ]:
#Find the number of rows in dataframe
df2.count()

In [ ]:
df3 = spark.read.options(delimiter='|', inferschema=True, header=True).csv("/content/file.txt")
df3.show()

In [ ]:
unique_df=df3.dropDuplicates()

In [ ]:
unique_df.show()
df3.show()

In [ ]:
#Try to avoid this way instaed go for below option
df2.show()
filtered_df = df2.filter(df2["Country Name"] =="India")
filtered_df.show()

In [ ]:
#Filter column
from pyspark.sql.functions import col
filtered_df = df2.filter(col("Country Name")=="India")
filtered_df.show()



### Ques1. Filter the colums where country is India and year is 2000 and 2005 using & and | operators

In [ ]:
unique_df.show()
filtered_df=unique_df.filter((col("Country Name")=="India") |((col("Year")==2000) | (col("Year")==2005)))
filtered_df.show()



In [ ]:
df2.orderBy("Value",ascending=False).show()


In [ ]:
df2.select("Value").orderBy("Value",ascending=False).show()

In [ ]:
from pyspark.sql.functions import col, count, sum, avg, min, max, mean

# Sample DataFrame
data = [("John", 1000),
        ("Alice", 2000),
        ("Bob", 1500),
        ("Alice", 3000)]

columns = ["Name", "Salary"]
df = spark.createDataFrame(data, schema=columns)

# Perform aggregation
df.agg(
    count(col("Name")).alias("TotalCount"),
    sum(col("Salary")).alias("TotalSalary"),
    avg(col("Salary")).alias("AverageSalary"),
    min(col("Salary")).alias("MinSalary"),
    max(col("Salary")).alias("MaxSalary"),
    mean(col("Salary")).alias("MeanSalary")
).show()


##filter,groupby, aggregation all in one function

In [ ]:
from pyspark.sql.functions import col, sum, count

data = [("John", "Sales", 1000),
        ("Alice", "HR", 2000),
        ("Bob", "Sales", 1500),
        ("Alice", "Sales", 3000)]

columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data, schema=columns)


grouped_df = df.filter(df["Department"] == "Sales").groupBy("Name").agg(
    sum(col("Salary")).alias("TotalSalary"),
    count(col("Name")).alias("EmployeeCount")
)
grouped_df.show()

In [ ]:
df2.show()
final_df=(df2.filter((col("Year")=="2000") | (col("Year")=="2003"))
    .groupBy("Country Name")
    .agg(count(col("Country Name")).alias("CountryCount"),
      sum(col("Value")).alias("ValueCount")))
final_df.show()

In [ ]:
df3.show()

In [ ]:
df_renamed = df3.withColumnRenamed("Country Name", "Country")
df_renamed.show()

In [ ]:
# Partition and shuffling
#Wwhere we can use rdd amd where dataset
job
shared folder
config file

Spark Folder structure
=>Jobs-Pyspark Code(Read data from diff files)(Dump data to warehouse)
=>Shared => scripts
=>Configs

Data Dumps=>db2,postgres to dataframe

Spark Properties
1.num executors
2. Executor memory 10gb
3. executor cores
4. Driver memory

## 19 March


In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
SparkSession.sparkContext.setLogLevel('Error')

In [ ]:
def func():
  df=spark.read.csv("/content/Foods.csv", header=True)

  logger.info("Successfully Created")

func()

In [ ]:
import logging

logging.getLogger().setLevel(logging.INFO)

def func():
    try:
        df = spark.read.csv("/content/Foods.csv", header=True)
        df.show()
        logging.info("Successfully created")
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
    finally:
      spark.stop()

# Call the function
func()

INFO:root:Successfully created


+-------+----------+-----+
|Food ID| Food Item|Price|
+-------+----------+-----+
|      1|     Sushi| 3.99|
|      2|   Burrito| 9.99|
|      3|      Taco| 2.99|
|      4|Quesadilla| 4.25|
|      5|     Pizza| 2.49|
|      6|     Pasta|13.99|
|      7|     Steak|24.99|
|      8|     Salad|11.25|
|      9|     Donut| 0.99|
|     10|     Drink| 1.75|
+-------+----------+-----+

